In [1]:
import os
os.chdir("..")

# Préparation 

## Imports

In [2]:
import pandas as pd

pd.options.display.max_columns = 100

In [3]:
from src.utils import get_all_schema_path
from tableschema import Schema

## Lecture et mise en forme

In [4]:
df = pd.read_csv("notebooks/Envoi_Dico Alimentation/Table_Variable_Modele_181214170701.csv", 
                 encoding="latin1", sep=';')
df = df.drop(columns=['Chemin', 'Unnamed: 21', 'Unnamed: 22'])

In [5]:
df.columns = ['produit', 
              'name_table', 
              'title_table', 
              'name', 
              'description',
              'famille_concept', 
              'length', 
              'type', 
              'nomenclature', 
              'elementaire_ou_calculee', 
              'classe_donnee', 
              'regle_gestion', 
              'observation_variable',
              'sensible_ou_medicale', 
              'type_objet', 
              'mode_acces',
              'observation_produit_variable', 
              'liaison', 
              'observation_cumul', 
              'referentiel']

In [6]:
df.head(2)

,produit,name_table,title_table,name,description,famille_concept,length,type,nomenclature,elementaire_ou_calculee,classe_donnee,regle_gestion,observation_variable,sensible_ou_medicale,type_objet,mode_acces,observation_produit_variable,liaison,observation_cumul,referentiel
0,IR_BEN_R,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,ASS_NIR_ANO,Matricule anonymisé de l'ouvreur de droits,Bénéficiaires,17,Caractère,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAS Guide,NaN,IR_BEN_RASS_NIR_ANO,\n0,NaN
1,IR_BEN_R,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_IDT_ANO,Identifiant bénéficiaire anonymisé,Bénéficiaires,18,Varchar,NaN,Calculée,NaN,NaN,NaN,NaN,NaN,SAS Guide,NaN,IR_BEN_RBEN_IDT_ANO,\n0,NaN


In [7]:
df.duplicated().sum()

176

In [8]:
df = df.drop_duplicates()

## Signification variable Liaison
combinaison du nom de table et variable, ou produit et …

In [9]:
mask = ((df.name_table + df.name != df.liaison) & 
        (df.produit + df.name != df.liaison) & 
        (df.produit + df.name_table != df.liaison)
       )
df[mask].liaison.str[:4].value_counts()

Series([], Name: liaison, dtype: int64)

## Nomenclatures

In [10]:
from src.utils import get_all_nomenclatures_csv_schema_path

nomenclatures_set = set(csv_path.split("/")[-1][:-4] for csv_path, _ in get_all_nomenclatures_csv_schema_path())

## Standardize values

In [11]:
def map_with_mapping(s):
    if s in mapping:
        return mapping[s]
    return s

### fillna et strip


In [12]:
df = df.fillna("")

In [13]:
for column in df.columns:
    df[column] = df[column].str.strip()

### name_table

In [14]:
df.name_table = df.name_table.str.upper()

### produit

In [15]:
mapping = {
    'CMDC': 'Causes de décès',
    'CARTO' : 'CARTOGRAPHIE_PATHOLOGIES',
    "IR_BEN_R" : "BENEFICIAIRE",
    "IR_IBA_R" : "BENEFICIAIRE",
    "IR_IMB_R" : "DCIR_DCIRS",
    "IR_ORC_R" : "DCIR_DCIRS",
    "IR_ETM_R" : "DCIR_DCIRS",
    "IR_ACS_R" : "DCIR_DCIRS",
    "IR_MAT_R" : "DCIR_DCIRS",
    "IR_MTT_R" : "DCIR_DCIRS"
}
df.produit = df.produit.map(lambda s: mapping[s] if s in mapping else s)
df.produit.value_counts()

CARTOGRAPHIE_PATHOLOGIES     711
DCIRS                        409
DCIR                         402
EGB                          390
EGBS                         377
Univers SNIREP               370
Univers BERF                 254
Univers AMOS-EXE-PRS         240
Univers DAMIR                168
Suivi actes de biologie      160
Univers AMOS-CDS             152
Univers AMOS_ODS_LPP         150
AMOS                         126
Univers AMOS-CCAM-Prest      125
Univers AMOS-CCAM-Associa    110
DAMIR                         77
BGNA                          75
Causes de décès               61
DCIR_DCIRS                    50
Univers UCD                   45
BENEFICIAIRE                  38
Suivi actes de pharmacie      31
Name: produit, dtype: int64

In [16]:
mapping = {
    'Référentiel médicalisé du SNIIRAM-SNDS': 'Référentiel médicalisé des bénéficiaires (ALD)',
    'Dépenses des 57 millions dassurés sélectionnés': "Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N",
    'Identifiants des 57 millions dassurés sélectionnés de lannée yyyy': "Table individus de la cartographie des pathologies pour l'année AAAA et l'algorithme N",
    'Caractérisation des 57 M dassurés pr 56 grp de pathologies': "Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N",    
}
df.title_table = df.title_table.map(lambda s: mapping[s] if s in mapping else s)

#### Carto

In [17]:
# => Vérification que toutes les tables sont identiques en fonction des années pour la carto

for table in ['DEP', 'IDE', 'IND']:
    for year1, year2 in [('2013', '2014'), ('2014', '2015')]:
        table1 = 'CT_{}_{}_G4'.format(table, year1)
        table2 = 'CT_{}_{}_G4'.format(table, year2)
        df1 = df[df.name_table == table1].reset_index(drop=True).drop(columns="name_table")
        df2 = df[df.name_table == table2].reset_index(drop=True).drop(columns="name_table")
        assert df1.equals(df2)

In [18]:
# Remise en forme du nom des tables
mask = df.name_table.str.startswith("CT_")
df.name_table[mask] = df[mask].name_table.str[:7] + 'AAAA_GN'
df = df.drop_duplicates()

#### name

In [19]:
df.name = df.name.str.rstrip("\xa0")

### referentiel

In [20]:
mapping = {
    'IR_IBA_R': 'IR_IBA_R',
    'référentiel des bénéficiaires du SNIIRAM IR_BEN_R' : 'IR_BEN_R',
    'DA_PRA_R': 'DA_PRA_R',
    'T_NMED - Référentiel des Médicaments': 'T_NMED',
    'RG_AR_DET_FT - DA_PRA_R': 'RG_AR_DET_FT - DA_PRA_R',
    'SNDA': 'SNDA',
    'Fichier National des Professionnels de Santé (FNPS)': 'FNPS',
}
df.referentiel = df.referentiel.map(map_with_mapping)
df.referentiel.value_counts()

                           3954
IR_IBA_R                     33
DA_PRA_R                     23
IR_BEN_R                     22
T_NMED                       10
SNDA                          2
RG_AR_DET_FT - DA_PRA_R       2
FNPS                          1
Name: referentiel, dtype: int64

### elementaire

In [21]:
mapping = {
    'Elémentaire': 'Élémentaire', 
    'Calculée': 'Calculée', 
    'Calculé': 'Calculée', 
    'Élémentaire': 'Élémentaire'
}
df.elementaire_ou_calculee = df.elementaire_ou_calculee.map(map_with_mapping)
df.elementaire_ou_calculee.value_counts()

               2896
Élémentaire     770
Calculée        381
Name: elementaire_ou_calculee, dtype: int64

### type

In [22]:
mapping = {
    "Numérique": "number",
    "Caractère": "string",
    "Date": "date",
    "numérique": "number",
    "caractère": "string",
    "VARCHAR2": "string",
    "Varchar": "string",
    "date": "date"
}
df.type = df.type.map(lambda s: mapping[s] if s in mapping else s)

df.type.value_counts()

string    1981
number    1772
date       278
            16
Name: type, dtype: int64

TODO :
- clé jointure


TODO :
- dates et formats de dates

### lenght

TODO :
- Extraire le nombre avant et après la virgule, 
- Attention a vérifier que ce sont bien des nombres non vides


In [23]:
mask = (df.length.str.contains('\.') | df.length.str.contains('\,'))
df[mask]

,produit,name_table,title_table,name,description,famille_concept,length,type,nomenclature,elementaire_ou_calculee,classe_donnee,regle_gestion,observation_variable,sensible_ou_medicale,type_objet,mode_acces,observation_produit_variable,liaison,observation_cumul,referentiel
98,DCIRS,NS_PRS_F,Table des prestations,RGO_REM_TAU,Taux de remboursement (part Régime Obligatoire),Prestations,"5,2",number,,Élémentaire,,,,,,SAS Guide,,DCIRSRGO_REM_TAU,0,
106,DCIRS,NS_CAM_F,Table simplifiée des données de codage de la C...,CAM_ACT_PRU,Prix unitaire CCAM de l'acte médical,Prestations,"7,2",number,,,,,,,,SAS Guide,,DCIRSCAM_ACT_PRU,0,
114,DCIRS,NS_CAM_F,Table simplifiée des données de codage de la C...,CAM_REM_BSE,Base de remboursement de la CCAM,Indicateurs relatifs aux dépenses,"7,2",number,,,,,,,,SAS Guide,,DCIRSCAM_REM_BSE,0,
137,DCIRS,NS_CAM_F,Table simplifiée des données de codage de la C...,RGO_REM_TAU,Taux de remboursement (part Régime Obligatoire),Prestations,"5,2",number,,Élémentaire,,,,,,SAS Guide,,DCIRSRGO_REM_TAU,0,
164,DCIRS,NS_DTR_F,Table simplifiée des données de codage de tran...,RGO_REM_TAU,Taux de remboursement (part Régime Obligatoire),Prestations,"5,2",number,,Élémentaire,,,,,,SAS Guide,,DCIRSRGO_REM_TAU,0,
166,DCIRS,NS_DTR_F,Table simplifiée des données de codage de tran...,TRS_ACT_PRU,Prix unitaire du code affiné du transport,Prestations,"10,5",number,IR_DTR_V,,,,,,,SAS Guide,,DCIRSTRS_ACT_PRU,0,
198,DCIRS,NS_INV_F,Table simplifiée des données de codage des inv...,RGO_REM_TAU,Taux de remboursement (part Régime Obligatoire),Prestations,"5,2",number,,Élémentaire,,,,,,SAS Guide,,DCIRSRGO_REM_TAU,0,
222,DCIRS,NS_PHA_F,Table simplifiée des données de codage de phar...,PHA_ACT_PRU,Prix unitaire du médicament codé en CIP,Prestations,"10,5",number,IR_PHA_R,,,,,,,SAS Guide,,DCIRSPHA_ACT_PRU,0,
225,DCIRS,NS_PHA_F,Table simplifiée des données de codage de phar...,PHA_DEC_PRU,Prix unitaire de l'unité délivrée,Indicateurs relatifs aux dépenses,"7,2",number,,,,,,,,SAS Guide,,DCIRSPHA_DEC_PRU,0,
237,DCIRS,NS_PHA_F,Table simplifiée des données de codage de phar...,RGO_REM_TAU,Taux de remboursement (part Régime Obligatoire),Prestations,"5,2",number,,Élémentaire,,,,,,SAS Guide,,DCIRSRGO_REM_TAU,0,


### nomenclatures

TODO 
- CT_IND_AAAA_GN : beaucoup d'erreurs a priori
- gérer nomenclatures multiples
- ne pas prendre les nouveaux ir_pha_r dans er_pha_r et ns_pha_r
- BE_IDE_R à traiter (on l'a pas, la récupérer ?)
- ER_TIP_F/NS_PRS_F : TIP_ACT_PRU -> pas IR_DTE_V (prix)
- ER_TIP_F/NS_PRS_F : TIP_PUB_PRX -> pas IR_PRF_V (prix)
- IR_IMB_R : INS_DTE -> pas IR_IMB_R
- NS_BIO_F : BIO_ACT_QSN -> pas IR_BIO_R (quantité)
- NS_PHA_F : PHA_ACT_PRU -> pas IR_PHA_R
- dans dcir simplifié remplacer partout IR_ACT_V, IR_SPE_V par IR_SPA_D

### description

TODO 
- suppression anomalie

In [24]:
df[df.description.str.contains("EPK")].description

827     Indicateur Référentiel EPK
1216    Indicateur Référentiel EPK
Name: description, dtype: object

In [25]:
for abbrev, nom in [
    ("Cpt", "comptable"),
    ("Regul", "régulation"),
    ("Dcpte", "décompte"),
    ("Prs", "Prestation"),
    ("Pharma", "pharmacie"),
    ("EPK","Entente Préalable kinésithérapie")

]:
    df.description = df.description.str.replace(abbrev, nom)

In [26]:
mapping = {
    "anomalie": "",
}
df.description = df.description.map(lambda s: mapping[s] if s in mapping else s)

## Préparation produits

In [27]:
produit_a_garder = [
    'CARTOGRAPHIE_PATHOLOGIES',
 'DCIR',
 'DCIRS',
 'Causes de décès',
 'DCIR_DCIRS',
 'BENEFICIAIRE',
 ]

df = df[df.produit.isin(produit_a_garder)]

In [28]:
df

,produit,name_table,title_table,name,description,famille_concept,length,type,nomenclature,elementaire_ou_calculee,classe_donnee,regle_gestion,observation_variable,sensible_ou_medicale,type_objet,mode_acces,observation_produit_variable,liaison,observation_cumul,referentiel
0,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,ASS_NIR_ANO,Matricule anonymisé de l'ouvreur de droits,Bénéficiaires,17,string,,,,,,,,SAS Guide,,IR_BEN_RASS_NIR_ANO,0,
1,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_IDT_ANO,Identifiant bénéficiaire anonymisé,Bénéficiaires,18,string,,Calculée,,,,,,SAS Guide,,IR_BEN_RBEN_IDT_ANO,0,
2,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_IDT_TOP,Top identifiant bénéficiaire Anonymisé,Bénéficiaires,1,number,,Calculée,,,,,,SAS Guide,,IR_BEN_RBEN_IDT_TOP,0,
3,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_CDI_NIR,Code d'identification du NIR,Bénéficiaires,2,string,IR_NIR_V,,,,,,,SAS Guide,,IR_BEN_RBEN_CDI_NIR,0,
4,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_DCD_AME,Année et mois de décès du bénéficiaire,Bénéficiaires,6,string,,,,,,,,SAS Guide,,IR_BEN_RBEN_DCD_AME,,
5,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_DCD_DTE,Date de décès du bénéficiaire,Bénéficiaires,,date,IR_DTE_V,,,,,donnée sensible,,SAS Guide,,IR_BEN_RBEN_DCD_DTE,,
6,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_DTE_INS,Date d'insertion dans le référentiel,Technique,,date,IR_DTE_V,,,,,,,SAS Guide,,IR_BEN_RBEN_DTE_INS,,
7,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_DTE_MAJ,Date de mise à jour,Technique,,date,IR_DTE_V,,,,,,,SAS Guide,,IR_BEN_RBEN_DTE_MAJ,,
8,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_NAI_ANN,Année de naissance du bénéficiaire,Bénéficiaires,4,string,,,,,,donnée sensible,,SAS Guide,,IR_BEN_RBEN_NAI_ANN,,
9,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_NAI_MOI,Mois de naissance du bénéficiaire,Bénéficiaires,2,string,,,,,,,,SAS Guide,,IR_BEN_RBEN_NAI_MOI,,


In [29]:
xdf = df[df.name_table == 'CT_IND_AAAA_GN']

In [ ]:
"IMB_ETM_NAT",
            "description": "Type de PEC (ALD, Invalidité ou ATMP)",

## Create descriptor

In [76]:
for i, (produit, name_table) in df[["produit", 'name_table']].drop_duplicates().iterrows():
    # Restriction table
    sdf = df[(df.produit == produit) & (df.name_table == name_table)]
    
    assert sdf.title_table.nunique() == 1
    title = sdf.title_table.unique()[0]

   
    # Lecture schema
    schema_path = "schemas/{}/{}.json".format(produit, name_table)
    assert os.path.exists(schema_path)
    schema = Schema(schema_path)

    
    # MAJ table
    schema.descriptor.update({
        "name": name_table, 
        "title": title, 
        "produit": produit
    })
    
    
    # Différences d'ensembles de variables
    schema_names = set([name.upper() for name in schema.field_names]) 
    dico_names = set(sdf.name)
    set_schema_dico = schema_names - dico_names
    set_dico_schema = dico_names - schema_names
      

    if set_schema_dico or set_dico_schema:
        print()
        print(produit, name_table, title) 
        if set_dico_schema:
            print("Variables absente dans le schéma", set_dico_schema)
    
    # MAJ variables
    for field in schema.descriptor['fields']:
        name = field['name']
        if not name.upper() in dico_names:
            print('Variable absente dans le dico cnam', name)
            continue
        
        columns_to_update = ['description', 'type', 'nomenclature', 'length']
        columns_to_update = ['description']
        record = sdf[sdf.name == name.upper()][columns_to_update].to_dict(orient="records")[0]
        for key in columns_to_update:
            if record[key] == "":
                record.pop(key)
        field.update(record)
    
    try:
        schema.commit(strict=True)
    except Exception as e:
        print(e.errors)
        raise e
    
    schema.save(schema_path, ensure_ascii=False)


DCIR_DCIRS IR_ACS_R Table historique des affiliations à l'aide à la complémentaire santé
Variables absente dans le schéma {'IND_RNM_BEN'}

DCIR_DCIRS IR_MTT_R Table historique des médecins traitants
Variable absente dans le dico cnam MTT_DEP_DTE
Variable absente dans le dico cnam MTT_FIN_DTE
Variable absente dans le dico cnam MTT_MTF_COD
Variable absente dans le dico cnam MTT_ORI_COD
Variable absente dans le dico cnam MTT_PFS_NUM

DCIRS NS_BIO_F Table simplifiée des données de codage de biologie
Variable absente dans le dico cnam RGO_REM_TAU

DCIRS NS_PRS_F Table des prestations
Variable absente dans le dico cnam BEN_RES_COM

DCIR ER_PRS_F Table des prestations
Variables absente dans le schéma {'BEN_ACS_TOP', 'PRS_MNT_MAJ', 'BEN_CTA_TYP', 'PRS_TYP_MAJ', 'EXE_CTX_BEN', 'BEN_DRT_SPF', 'PRS_DIS_PRE', 'CPL_FTA_COD', 'PRS_PPU_SEC', 'PRS_DRA_AME', 'PRS_QTT_MAJ', 'PRE_IND_PEL', 'DRG_AFF_NAT', 'EXE_CTX_PFS', 'CPL_REM_TAU'}
Variable absente dans le dico cnam PFS_EXE_NUMC
Variable absente dans 